# Loading to docker image of ElasticSearch

In [ ]:
from etl_script import *
import requests
import json

In [ ]:
codes_folder = "../ReadFiles/Codigos"

codes, entries = get_files(codes_folder, return_entries=True)

c = 0
for code, entry in zip(codes,entries):
    if c < 2:
        print(entry, '|', code[0])
        print(code[1:3])
        print("\n-------------------------------\n")
    c += 1

## ID of legal sources

The ID of every legal source (codes, statutes, norms) are storage on the csv file "codes_id". The ID were made "by hand" in order to improve the human reading.

In [ ]:
import csv

codesid_folder = "../ReadFiles/codes_id.csv"

codesid_path = os.path.abspath(codesid_folder)
with open(codesid_path, 'r', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    codes_id = {}
    for row in csv_reader:
        codes_id[row[0]] = row[1]

    csv_file.close()

# codes_id

## New hierarchy

This hierarchy was made according to observed on the research 

In [ ]:
hierarchy = {
    # Agregados
    'LIBRO': 'book',
    'PARTE': 'part',
    
    # Original
    'TITULO' : 'headline',
    'DISPOSICIONES' : 'headline',
    'CAPITULO' : 'chapter',
    
    # Agregado
    'SECCION': 'section',
    
    #Original
    'ARTICULO' : 'article',
    
    # Agregados
    'CONSIDERANDO' : 'article',
    'PREAMBULO' : 'article',
}

## Look the content of the every legal source

Here you will find the ammount of each hierarchy that every file has on it.

In [ ]:
# show_codes_content(codes, entries, hierarchy)

## Watch the evolution of every change in model.

For the embeddings model, a delimitation by dots and dot-comma was made. Here you can see the evolution on the quantity of elements for each interation by passing a True value on "show_difference"

In [ ]:
create_json_files(codes, entries, codes_id, show_difference=False)

In [ ]:
main_url = "http://localhost:9200/test_all"

# Loading to docker
## Delete on case of emergency

During our development, the cases of overloading were common. Eviting to have more articles than the needed, a delete process was stablished on the beginning. If you need it, just uncomment the next cell

In [ ]:
# response = requests.delete(main_url)
# response.text

## Mapping

Gives the structure of each document to Elastic Search. Like initializing a Data Base on SQL, the elements on the doc needs to respect the value set on the mapping.json file.

In [ ]:
with open('mapping.json', 'r') as json_file:
    mapping = json.load(json_file)
    json_file.close()

    response = requests.put(main_url, json=mapping)
response.text

## Check mapping

Just a revision that everything is in order.

In [ ]:
response = requests.get(main_url)
response.text

## Load of embedds vector

This is where the ML magic begings. Don't worry, it takes a time.

In [ ]:
import pandas as pd
from fastparquet import ParquetFile

pf = ParquetFile('cons_codes.parq')

df = pf.to_pandas()
df = df[['id', 'embedds']]

embedds = df['embedds'].tolist()
ids = df['id'].tolist()

embeddings = {}
for art_id, embedd in zip(ids, embedds):
    embeddings[art_id] = embedd
    
# embeddings['336transporte00000201000001']

## The loadding

If you want to see the process, go to etl_script.

In [ ]:
codes_folder = "../ReadFiles/Embeddings"
load_to_es_docker(codes_folder)

## Counting of total values at the docker image.

In [ ]:
count_docs = f"{main_url}/_count?q=*"
count_test = requests.get(count_docs)

result = json.loads(count_test.text)
result

# Query to Elastic Search
## Simple Query

In [ ]:
local_test = f"{main_url}/_search"
query_test = {
    "query": {
        "simple_query_string": {
            "query": "constitucion"
        }
    }
}
query_test = requests.get(local_test, json=query_test)

result = json.loads(query_test.text)
print(result)
print(result['hits']['total']['value'])
print(result['hits']['max_score'])
best_rated = result['hits']['hits'][0]


## Tensorflow + ElasticSearch = Embeddings model

In [ ]:
!pip3 install tensorflow_hub
!pip3 install tensorflow_text

In [ ]:
from tf_model import embed_text
import requests

local_test = "http://localhost:9200/test_all/_search"
query = "adolescencia"

query_vector = embed_text(query)

query_test = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
      },
      "script": {
        "source": "cosineSimilarity(params.queryVector, doc['embedds'])",
        "params": {
          "queryVector": query_vector
        }
      }
    }
  }
}

query_test = requests.get(local_test, json=query_test)
result = json.loads(query_test.text)
print(result['hits']['total']['value'])
print(result['hits']['max_score'])
best_rated = result['hits']['hits'][1]
best_rated